# autogloun 라이브러리를 이용한 분류

우선 너무 단순하게 전처리도 안하고 나온 결과인데 1등을 해서 놀랬네요,,, 최신 기법들을 찾아보고 적용해보는 것이 제일 중요한 것 같습니다.

autogloun의 경우 장점으로
1. robust
2. 분류든 회귀든 훈련 가능
3. 시간을 설정할 수 있다는 점입니다.

또한 특이점으로 직접 저희가 앙상블하고 스태킹 안해도 autoloun내에서 알아서 해준다는 점이 크다는 점입니다!

https://auto.gluon.ai/stable/tutorials/tabular_prediction/tabular-indepth.html
자세한 튜닝 방법은 위 사이트를 참고하시면 좋을 것 같습니다.

#### autoluon 라이브러리 설치해주기

In [ ]:
!pip install autogluon

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 49.8/49.8 kB ? eta 0:00:00
     ---------------------------------------- 60.0/60.0 kB 3.1 MB/s eta 0:00:00
     -------------------------------------- 303.4/303.4 kB 9.5 MB/s eta 0:00:00
     -------------------------------------- 103.6/103.6 kB 5.8 MB/s eta 0:00:00
     -------------------------------------- 226.5/226.5 kB 6.8 MB/s eta 0:00:00
     -------------------------------------- 292.5/292.5 kB 9.1 MB/s eta 0:00:00
     ---------------------------------------- 62.1/62.1 kB ? eta 0:00:00
     ------------------------------------- 802.2/802.2 kB 10.0 MB/s eta 0:00:00
     ---------------------------------------- 44.7/44.7 kB ? eta 0:00:00
     ---------------------------------------- 7.6/7.6 MB 11.3 MB/s eta 0:00:00
     ---------------------------------------- 1.0/1.0 MB 9.3 MB/s eta 0:00:00
     --------------------------------------- 20.7/20.7 MB 11.1 MB/

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t

     -------------------------------------- 109.4/109.4 kB 6.2 MB/s eta 0:00:00
     ------------------------------------- 409.8/409.8 kB 12.9 MB/s eta 0:00:00
     ------------------------------------- 410.8/410.8 kB 12.9 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 10.6 MB/s eta 0:00:00
     -------------------------------------- 113.5/113.5 kB 6.9 MB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 11.0 MB/s eta 0:00:00
     -------------------------------------- 240.9/240.9 kB 7.2 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 9.5 MB/s eta 0:00:00
     ---------------------------------------- 7.0/7.0 MB 11.4 MB/s eta 0:00:00
     ------------------------------------- 601.9/601.9 kB 12.6 MB/s eta 0:00:00
     ---------------------------------------- 1.3/1.3 MB 11.5 MB/s eta 0:00:00
     -------------------------------------- 102.4/102.4 kB 5.8 MB/s eta 0:00:00
     --------------------------------------- 3

     ---------------------------------------- 1.3/1.3 MB 10.2 MB/s eta 0:00:00
     ------------------------------------- 181.6/181.6 kB 10.7 MB/s eta 0:00:00
     -------------------------------------- 126.5/126.5 kB 7.3 MB/s eta 0:00:00
     -------------------------------------- 781.3/781.3 kB 9.9 MB/s eta 0:00:00
     ---------------------------------------- 84.5/84.5 kB 4.6 MB/s eta 0:00:00
     -------------------------------------- 468.5/468.5 kB 9.8 MB/s eta 0:00:00
     ---------------------------------------- 7.0/7.0 MB 10.4 MB/s eta 0:00:00
     -------------------------------------- 151.7/151.7 kB 9.4 MB/s eta 0:00:00
  Created wheel for fairscale: filename=fairscale-0.4.6-py3-none-any.whl size=307237 sha256=f584ca780158f707b74a41914f156073071e6bb6dd45fa867fd71aa0ef03dfda
  Stored in directory: c:\users\zzzss\appdata\local\pip\cache\wheels\8d\7b\f9\6c7a350821ca240450550801e17996aee846b71caaccda7a32
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-

### TablularPredictor 와 TabularDataset 불러오기

In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor
train_data = TabularDataset('./train.csv')
test_data = TabularDataset('./test.csv')

ID 컬럼은 필요없다고 생각하여 drop 하여 진행하였습니다.

In [2]:
train_data.drop('ID',axis = 1, inplace = True)
test_data.drop('ID',axis = 1, inplace = True)

목표 타겟 값은 'genre' 이며 평가 지표는 f1 macro 입니다.

이 라이브러리에서는 시간 제한을 걸어줄 수 있습니다.  저는 3시간으로 걸겠습니다. (더 오래 걸면 효과가 더 잘 나올 수도 있을 지 모르겠습니다.)

In [3]:
label = 'genre'
eval_metric = 'f1_macro'
time_limit = 3600 * 3

만일 위에서 drop 한 것을 까먹으셨다면  learner_kwargs={'ignored_columns': ['ID']} 이것을 추가해주셔도 됩니다.

best_quality로 하면 제일 좋은 지표가 나오기에 사용해줬습니다. 이외에 대한 튜닝 작업은 위 사이트에 자세히 적혀있으니 참고해주시기 바랍니다.

In [4]:
predictor = TabularPredictor(
    label=label, eval_metric=eval_metric
).fit(train_data, presets='best_quality', time_limit=time_limit)

No path specified. Models will be saved in: "AutogluonModels\ag-20230130_141640\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 10800s
AutoGluon will save models to "AutogluonModels\ag-20230130_141640\"
AutoGluon Version:  0.6.2
Python Version:     3.9.12
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Train Data Rows:    25383
Train Data Columns: 11
Label Column: genre
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == object).
	First 10 (of 15) unique label values:  ['trap', 'techno', 'Trap Metal', 'Dark Trap', 'Rap', 'Underground Rap', 'hardstyle', 'RnB', 'trance', 'techhouse']
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multicl

	3.1s	 = Validation runtime
Fitting model: LightGBMXT_BAG_L1 ... Training model for up to 5111.17s of the 8712.88s of remaining time.
	Fitting 8 child models (S3F1 - S3F8) | Fitting with ParallelLocalFoldFittingStrategy
	0.6513	 = Validation score   (f1_macro)
	144.82s	 = Training   runtime
	56.77s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ... Training model for up to 5061.16s of the 8662.87s of remaining time.
	Fitting 8 child models (S3F1 - S3F8) | Fitting with ParallelLocalFoldFittingStrategy
	0.65	 = Validation score   (f1_macro)
	103.17s	 = Training   runtime
	18.23s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ... Training model for up to 5023.01s of the 8624.72s of remaining time.
	Fitting 8 child models (S3F1 - S3F8) | Fitting with ParallelLocalFoldFittingStrategy
	0.6514	 = Validation score   (f1_macro)
	1253.17s	 = Training   runtime
	0.81s	 = Validation runtime
Fitting model: XGBoost_BAG_L1 ... Training model for up to 4602.75s of the 8204.46s of remaining

	Fitting 8 child models (S7F1 - S7F8) | Fitting with ParallelLocalFoldFittingStrategy
	0.6466	 = Validation score   (f1_macro)
	418.18s	 = Training   runtime
	6.42s	 = Validation runtime
Fitting model: LightGBMXT_BAG_L1 ... Training model for up to 1737.63s of the 5339.34s of remaining time.
	Fitting 8 child models (S7F1 - S7F8) | Fitting with ParallelLocalFoldFittingStrategy
	0.6532	 = Validation score   (f1_macro)
	309.89s	 = Training   runtime
	140.54s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ... Training model for up to 1682.12s of the 5283.84s of remaining time.
	Fitting 8 child models (S7F1 - S7F8) | Fitting with ParallelLocalFoldFittingStrategy
	0.6538	 = Validation score   (f1_macro)
	210.09s	 = Training   runtime
	36.52s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ... Training model for up to 1647.28s of the 5248.99s of remaining time.
	Fitting 8 child models (S7F1 - S7F8) | Fitting with ParallelLocalFoldFittingStrategy
	0.6523	 = Validation score   (f1_ma

어떤 모델이 제일 잘 나왔는지 확인하는 방법은 leaderboard 를 넣어주시면 점수 내림차순으로 모델을 보여줍니다.

In [5]:
predictor.leaderboard(silent = True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,0.691686,379.206497,9417.314937,0.015630,17.112079,3,True,25
1,XGBoost_BAG_L2,0.686341,350.765159,6050.470530,1.620220,370.935474,2,True,23
2,NeuralNetFastAI_BAG_L2,0.685797,352.567996,5745.546289,3.423057,66.011233,2,True,15
3,LightGBMXT_BAG_L2,0.683876,350.891349,5797.625450,1.746410,118.090394,2,True,16
4,LightGBM_BAG_L2,0.683599,360.964065,6161.640260,11.819126,482.105204,2,True,17
5,CatBoost_BAG_L2,0.681313,349.483991,8662.105322,0.339052,2982.570266,2,True,20
6,LightGBMLarge_BAG_L2,0.680375,363.503419,5950.512889,14.358480,270.977833,2,True,24
7,ExtraTreesGini_BAG_L2,0.679440,352.640018,5684.707055,3.495080,5.171999,2,True,21
8,ExtraTreesEntr_BAG_L2,0.678448,352.682095,5684.657621,3.537156,5.122565,2,True,22
9,RandomForestGini_BAG_L2,0.677962,352.417822,5699.413488,3.272883,19.878432,2,True,18


get_model_best를 이용하면 제일 좋은 모델을 사용하게 됩니다.

이제 test data에 넣어 predict를 해주시면 됩니다.

In [6]:
model_to_use = predictor.get_model_best()
model_pred = predictor.predict(test_data, model=model_to_use)

나온 결과 값을 submission 파일에 넣어주시면 끝입니다.

In [7]:
import pandas as pd
submission = pd.read_csv('./sample_submission.csv')

In [8]:
submission["genre"] = model_pred
submission.head()

,ID,genre
0,TEST_00000,Hiphop
1,TEST_00001,Underground Rap
2,TEST_00002,RnB
3,TEST_00003,techhouse
4,TEST_00004,Pop


In [ ]:
submission.to_csv('./submission.csv', index=False)

만일 feature importance를 확인하고 싶다면 아래와 같이 작성하시면 됩니다.

predictor.feature_importance(test_data)

#### 번외

이렇게 auto_stack으로 stacking 방식을 이용해 넣는 방식도 있으나 위 점수보다 결과가 안좋기에 제외하였습니다.

In [ ]:
predictor = TabularPredictor(
    label=label, eval_metric=eval_metric
).fit(train_data, presets='best_quality', time_limit=time_limit,auto_stack=True)

No path specified. Models will be saved in: "AutogluonModels\ag-20230124_031651\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 10800s
AutoGluon will save models to "AutogluonModels\ag-20230124_031651\"
AutoGluon Version:  0.6.2
Python Version:     3.9.12
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Train Data Rows:    25383
Train Data Columns: 11
Label Column: genre
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == object).
	First 10 (of 15) unique label values:  ['trap', 'techno', 'Trap Metal', 'Dark Trap', 'Rap', 'Underground Rap', 'hardstyle', 'RnB', 'trance', 'techhouse']
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multicl

	3.52s	 = Validation runtime
Fitting model: LightGBMXT_BAG_L1 ... Training model for up to 4322.19s of the 7923.95s of remaining time.
	Fitting 8 child models (S3F1 - S3F8) | Fitting with ParallelLocalFoldFittingStrategy
	0.6513	 = Validation score   (f1_macro)
	167.27s	 = Training   runtime
	66.59s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ... Training model for up to 4244.63s of the 7846.39s of remaining time.
	Fitting 8 child models (S3F1 - S3F8) | Fitting with ParallelLocalFoldFittingStrategy
	0.65	 = Validation score   (f1_macro)
	119.57s	 = Training   runtime
	21.84s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ... Training model for up to 4186.99s of the 7788.75s of remaining time.
	Fitting 8 child models (S3F1 - S3F8) | Fitting with ParallelLocalFoldFittingStrategy
	0.6514	 = Validation score   (f1_macro)
	1975.33s	 = Training   runtime
	1.7s	 = Validation runtime
Fitting model: XGBoost_BAG_L1 ... Training model for up to 3414.04s of the 7015.8s of remaining 

	21.48s	 = Training   runtime
	0.46s	 = Validation runtime
Completed 1/20 k-fold bagging repeats ...
Fitting model: WeightedEnsemble_L3 ... Training model for up to 360.0s of the -9.2s of remaining time.
	0.6897	 = Validation score   (f1_macro)
	17.71s	 = Training   runtime
	0.03s	 = Validation runtime
AutoGluon training complete, total runtime = 10827.04s ... Best model: "WeightedEnsemble_L3"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels\ag-20230124_031651\")


In [ ]:
best_model = predictor.get_model_best()
print(best_model)
model_pred = predictor.predict(test_data, model=best_model)
predictor.leaderboard(silent = True)

WeightedEnsemble_L3


,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,0.689741,238.517963,8899.100524,0.031373,17.707584,3,True,25
1,LightGBM_BAG_L2,0.685145,210.568462,8134.721413,16.688136,1063.340639,2,True,17
2,XGBoost_BAG_L2,0.685073,195.844577,7261.281245,1.964251,189.900471,2,True,23
3,NeuralNetFastAI_BAG_L2,0.684607,199.647408,7185.073359,5.767082,113.692585,2,True,15
4,LightGBMXT_BAG_L2,0.681716,207.168833,7468.562696,13.288507,397.181922,2,True,16
5,RandomForestGini_BAG_L2,0.679059,196.950859,7090.716340,3.070533,19.335566,2,True,18
6,ExtraTreesGini_BAG_L2,0.678895,197.249639,7076.460935,3.369313,5.080161,2,True,21
7,RandomForestEntr_BAG_L2,0.676076,196.951420,7108.283050,3.071094,36.902275,2,True,19
8,ExtraTreesEntr_BAG_L2,0.675356,196.918902,7076.137112,3.038576,4.756338,2,True,22
9,CatBoost_BAG_L2,0.674008,194.264943,8094.635385,0.384617,1023.254611,2,True,20


In [ ]:
import pandas as pd
submission = pd.read_csv('./sample_submission.csv')
submission["genre"] = best_model
submission.head()
submission.to_csv('./auto.csv', index=False)

너무 단순하게 결과가 바로 나와 최신 트랜드의 라이브러리 파악 중요성을 본 대회를 통해서 깨닫게 된 것 같습니다.

autogluon은 저희가 사용한 TabularPredictor 이외에도 multimodel, time series, image, object predicion에도 우수한 성능을 보이는 것으로 보입니다.

#### 이렇게 시간을 늘리면 성능이 더 향상이 됩니다.

In [9]:
label = 'genre'
eval_metric = 'f1_macro'
time_limit = 3600 * 5

In [10]:
predictor = TabularPredictor(
    label=label, eval_metric=eval_metric
).fit(train_data, presets='best_quality', time_limit=time_limit)

No path specified. Models will be saved in: "AutogluonModels\ag-20230130_172046\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 18000s
AutoGluon will save models to "AutogluonModels\ag-20230130_172046\"
AutoGluon Version:  0.6.2
Python Version:     3.9.12
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Train Data Rows:    25383
Train Data Columns: 11
Label Column: genre
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == object).
	First 10 (of 15) unique label values:  ['trap', 'techno', 'Trap Metal', 'Dark Trap', 'Rap', 'Underground Rap', 'hardstyle', 'RnB', 'trance', 'techhouse']
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multicl

	166.05s	 = Training   runtime
	2.58s	 = Validation runtime
Fitting model: LightGBMXT_BAG_L1 ... Training model for up to 10166.54s of the 16169.47s of remaining time.
	Fitting 8 child models (S3F1 - S3F8) | Fitting with ParallelLocalFoldFittingStrategy
	0.6513	 = Validation score   (f1_macro)
	108.14s	 = Training   runtime
	43.6s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ... Training model for up to 10117.34s of the 16120.27s of remaining time.
	Fitting 8 child models (S3F1 - S3F8) | Fitting with ParallelLocalFoldFittingStrategy
	0.65	 = Validation score   (f1_macro)
	81.03s	 = Training   runtime
	13.88s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ... Training model for up to 10081.45s of the 16084.38s of remaining time.
	Fitting 8 child models (S3F1 - S3F8) | Fitting with ParallelLocalFoldFittingStrategy
	0.6514	 = Validation score   (f1_macro)
	1212.91s	 = Training   runtime
	0.97s	 = Validation runtime
Fitting model: XGBoost_BAG_L1 ... Training model for up to 9

	Fitting 8 child models (S7F1 - S7F8) | Fitting with ParallelLocalFoldFittingStrategy
	0.6466	 = Validation score   (f1_macro)
	388.0s	 = Training   runtime
	6.14s	 = Validation runtime
Fitting model: LightGBMXT_BAG_L1 ... Training model for up to 6756.21s of the 12759.14s of remaining time.
	Fitting 8 child models (S7F1 - S7F8) | Fitting with ParallelLocalFoldFittingStrategy
	0.6532	 = Validation score   (f1_macro)
	276.32s	 = Training   runtime
	124.49s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ... Training model for up to 6695.86s of the 12698.79s of remaining time.
	Fitting 8 child models (S7F1 - S7F8) | Fitting with ParallelLocalFoldFittingStrategy
	0.6538	 = Validation score   (f1_macro)
	189.16s	 = Training   runtime
	32.31s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ... Training model for up to 6657.77s of the 12660.71s of remaining time.
	Fitting 8 child models (S7F1 - S7F8) | Fitting with ParallelLocalFoldFittingStrategy
	0.6523	 = Validation score   (f1_

	672.51s	 = Training   runtime
	163.76s	 = Validation runtime
Repeating k-fold bagging: 11/20
Fitting model: NeuralNetFastAI_BAG_L1 ... Training model for up to 3303.87s of the 9306.8s of remaining time.
	Fitting 8 child models (S11F1 - S11F8) | Fitting with ParallelLocalFoldFittingStrategy
	0.6479	 = Validation score   (f1_macro)
	612.72s	 = Training   runtime
	9.7s	 = Validation runtime
Fitting model: LightGBMXT_BAG_L1 ... Training model for up to 3238.67s of the 9241.6s of remaining time.
	Fitting 8 child models (S11F1 - S11F8) | Fitting with ParallelLocalFoldFittingStrategy
	0.653	 = Validation score   (f1_macro)
	425.55s	 = Training   runtime
	204.54s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ... Training model for up to 3176.65s of the 9179.58s of remaining time.
	Fitting 8 child models (S11F1 - S11F8) | Fitting with ParallelLocalFoldFittingStrategy
	0.6539	 = Validation score   (f1_macro)
	300.7s	 = Training   runtime
	50.29s	 = Validation runtime
Fitting model: CatBo

	0.44s	 = Validation runtime
Fitting model: ExtraTreesGini_BAG_L2 ... Training model for up to 1175.62s of the 1175.47s of remaining time.
	0.681	 = Validation score   (f1_macro)
	5.66s	 = Training   runtime
	3.46s	 = Validation runtime
Fitting model: ExtraTreesEntr_BAG_L2 ... Training model for up to 1165.41s of the 1165.26s of remaining time.
	0.6785	 = Validation score   (f1_macro)
	5.1s	 = Training   runtime
	3.56s	 = Validation runtime
Fitting model: XGBoost_BAG_L2 ... Training model for up to 1155.56s of the 1155.4s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy
	0.6883	 = Validation score   (f1_macro)
	498.22s	 = Training   runtime
	3.52s	 = Validation runtime
Fitting model: NeuralNetTorch_BAG_L2 ... Training model for up to 644.97s of the 644.83s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy
	0.673	 = Validation score   (f1_macro)
	91.14s	 = Training   runtime
	3

In [11]:
predictor.leaderboard(silent = True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,0.693509,667.799559,11288.236923,0.030729,19.010508,3,True,26
1,XGBoost_BAG_L2,0.688269,647.208398,10881.349360,3.516779,498.221078,2,True,23
2,LightGBM_BAG_L2,0.686630,648.043972,10676.141302,4.352353,293.013020,2,True,17
3,LightGBMLarge_BAG_L2,0.685028,653.188308,10828.818218,9.496689,445.689936,2,True,25
4,NeuralNetFastAI_BAG_L2,0.684534,647.263077,10456.866000,3.571458,73.737719,2,True,15
5,LightGBMXT_BAG_L2,0.683158,646.903845,10542.132931,3.212226,159.004649,2,True,16
6,CatBoost_BAG_L2,0.681037,644.134806,15096.809029,0.443187,4713.680747,2,True,20
7,ExtraTreesGini_BAG_L2,0.680966,647.149879,10388.788186,3.458260,5.659904,2,True,21
8,RandomForestGini_BAG_L2,0.680358,647.183321,10405.085050,3.491702,21.956768,2,True,18
9,ExtraTreesEntr_BAG_L2,0.678521,647.246764,10388.225168,3.555145,5.096886,2,True,22


만일 시간이 오래 걸리는 것이 있다면 다음과 같이 제외를 시키고 진행해도 됩니다.

In [ ]:
excluded_model_types = ['NN_Torch','FASTAI']
predictor2 = TabularPredictor(
    label=label, eval_metric=eval_metric
).fit(train_data, presets='best_quality', time_limit=time_limit,excluded_model_types=excluded_model_types)